In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
def one_hot_encode(labels):
  n_labels = len(labels)
  n_unique_labels =len(np.unique(labels))
  one_hot_encode = np.zeros((n_labels,n_unique_labels))
  one_hot_encode[np.arange(n_labels),labels]=1
  return one_hot_encode


In [ ]:
dataset = pd.read_csv('/kaggle/input/bank-note-authentication-uci-data/BankNote_Authentication.csv')

In [ ]:
dataset.columns

In [ ]:
features = dataset[dataset.columns[0:4]].values
class_label = dataset[dataset.columns[4]].values
Y = one_hot_encode(class_label)

In [ ]:
features

In [ ]:
X,Y = shuffle(features,Y,random_state=1)

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(X,Y,test_size=0.2,random_state=42)
print(train_x.shape)
print(test_x.shape)

In [ ]:
learning_rate =0.3
training_epochs = 1000
cost_history = np.empty([1],dtype=float)
n_dim = X.shape[1]
n_class=2

n_hidden_1=60
n_hidden_2=60
n_hidden_3=60
n_hidden_4=60
print(n_dim)


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
x = tf.placeholder(tf.float32,[None,n_dim])
w = tf.Variable(tf.zeros([n_dim,n_class]))
b =tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32,[None,n_class])


In [ ]:
def multilayer_perceptron(x,weights,biases):
  layer1 = tf.add(tf.matmul(x,weights['h1']),bias['b1'])
  layer1 = tf.nn.sigmoid(layer1)

  layer2 = tf.add(tf.matmul(layer1,weights['h2']),bias['b2'])
  layer2 = tf.nn.sigmoid(layer2)

  layer3 = tf.add(tf.matmul(layer2,weights['h3']),bias['b3'])
  layer3 = tf.nn.sigmoid(layer3)

  layer4 = tf.add(tf.matmul(layer3,weights['h4']),bias['b4'])
  layer4 = tf.nn.relu(layer4)
  
  out_layer = tf.matmul(layer4,weights['out'])+ bias['out']

  return (out_layer)

In [ ]:
weights={
    'h1': tf.Variable(tf.truncated_normal([n_dim,n_hidden_1])),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2])),
    'h3':tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3])),
    'h4':tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4])),
    'out':tf.Variable(tf.truncated_normal([n_hidden_4,n_class]))
}

In [ ]:
bias ={
    'b1':tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2':tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3':tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4':tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out':tf.Variable(tf.truncated_normal([n_class]))
}

In [ ]:
init = tf.global_variables_initializer()

y = multilayer_perceptron(x,weights,bias)


In [ ]:
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y,labels=y_))
training_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost_function) 
sess = tf.Session()
sess.run(init)
mse_history=[]
accuracy_history=[]

In [ ]:
for epoch in range(training_epochs):
  sess.run(training_step,feed_dict={x:train_x,y_:train_y})
  cost = sess.run(cost_function,feed_dict={x:train_x,y_:train_y})
  cost_history = np.append(cost_history,cost)
  correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
  pred_y = sess.run(y,feed_dict={x:test_x})
  mse = tf.reduce_mean(tf.square(pred_y-test_y))
  mse_ = sess.run(mse)
  mse_history.append(mse_)
  accuracy = sess.run(accuracy,feed_dict={x:train_x,y_:train_y})
  accuracy_history.append(accuracy)
  print('epoch -',epoch,' -',' cost',cost,' - MSE',mse,' -train_accuracy',accuracy)


In [ ]:
plt.plot(range(1000),accuracy_history,'r')
plt.show()

In [ ]:
orrect_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print('Test accuracy',sess.run(accuracy,feed_dict={x:test_x,y_:test_y}))
pred_y = sess.run(y,feed_dict={x:test_x})
mse = tf.reduce_mean(tf.square(pred_y-test_y))
print('Mse:',sess.run(mse))
